# CSPB 3155 : Assignment 5


Topics Covered:
  - Let binding semantics
  - Scopes
  - Function calls

__Taylor Larrechea__

Note: Please first run the `TEST HELPER` cell that defines the `passed` function below. Failing to run this cell will make it hard for you to check your work.

In [1]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"Tests Passed (1 point)")
    else print(s"Tests Passed ($points points)") 
}

cell1.sc:108: procedure syntax is deprecated: instead, add `: Unit =` to explicitly declare `passed`'s return type [quickfixable]
def passed(points: Int) {
                        ^


defined function passed

## Problem 1: Multiple Simultaneous Let Bindings (15 points)

In class, we studied let bindings that assigned a "single" identifier to a "single" expression. Here, we will extend lettuce with multiple let bindings at the same time:

## Example 1 
~~~
let (x, y, z) = (10, 25.6, 30.3) in 
   x - y * z
~~~

The program computes `10 - 25.6 * 30.3`.

## Example 2

 In multi-let binding, we treat all the assignments as happening "simultaneously". For instance, the program 

~~~
let (x, y, z) = (10, x, y - x) in 
  x - y * z
~~~

is disallowed since neither `x` nor `y` are in scope in the right hand side of the multi-let binding. Example 2 above produces an `error` value since `x` and `y` are out of scope on the right hand side of the assignment.

## Example 3

~~~
let x = 15 in 
  let (x, y, z) = (x*x, -10 *x, -2*x) in 
     x + y + z
~~~

Note that the usage `x*x`, `-10*x` and `-2*x` refer back to `let x = 15` definition. However, the usages `x+y+z` refer to the result of the "multi-let" binding. Verify that this program will evaluate to "45".

## Grammar of Lettuce

Let us extend a minimalistic subset of Lettuce by adding a `MultiLet` statement as shown below.
$$\newcommand\Expr{\mathbf{Expr}}$$

$$\begin{array}{rcll}
  \Expr & \Rightarrow & \text{Const}(\mathbf{Double}) \\
  & | & \text{Ident}(\mathbf{String}) \\
  & | & \text{Plus}(\mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{Mult}(\mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{Let}(\mathbf{Ident}, \mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{MultiLet}(\mathbf{Ident}*, \mathbf{Expr}*, \mathbf{Expr}) & \leftarrow\ \text{ let (x1, .., xn) = (e1, ...,en) in e } \\
  & & & \text{Note: Number of identifiers n must match number of expressions n, or else evaluate to error }\\
  \end{array}$$
  
The scala definitions are given below.

In [2]:
sealed trait Expr
case class Const(d: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr 
case class Let(id: String, e1: Expr, e2: Expr) extends Expr
case class MultiLet(id: List[String], eList: List[Expr], e2: Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Mult
defined class Let
defined class MultiLet

## Semantics for MultiLet

$$\newcommand\semrule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}(\text{#3})} $$

Let us write down the semantic rules for a multilet statement:

$$\newcommand\eval{\textit{eval}}$$
$$\semrule{ \eval(\texttt{ei}, \texttt{env})= v_i,\ v_i \not= \mathbf{error}, \text{for}\ i = 1, \ldots, n,\ \texttt{newenv} = env \circ \{ \texttt{x1} \mapsto v_1, \ldots, \texttt{xn} \mapsto v_n \} }{ \eval( \texttt{MultiLet([x1,..,xn], [e1,...,en], e2), env}) = \eval(\texttt{e2, newenv})}{multilet-ok}$$

The semantic rule above tells you to 
  - Evaluate each of the expressions from `e1`, ..., `en` under the environment `env`.
  - Next, if all the expressions above evaluated without an error, it tells you to update the map `env` by binding each `xi` to $v_i$, the result of evaluating `ei`. You can use the Scala Map "++" operator to achieve this in one shot.
  - Finally, you should evaluate `e2` under the new environment created.

For convenience, we write a single "generic" semantic rule that shows that if some argument `ej` evaluates to an error, the whole expression is erroneous.

$$\semrule{ \eval(\texttt{ei}, \texttt{env})= v_i,\ v_i \not= \mathbf{error}, \text{for}\ i = 1, \ldots, j-1,\ \ eval(\texttt{ej}, \texttt{env})= \mathbf{error} }{ \eval( \texttt{MultiLet([x1,..,xn], [e1,...,en], e2), env}) = \mathbf{error}}{multilet-err-j}$$

### Interpreter for MultiLet Statements

Implement an interpreter for the lettuce language with `multi-let` statements. Your interpreter does not need to "propagate" error: instead you should throw an `IllegalArgumentException` whenever an error is encountered. 

### Style Guide

Use of var/while/for loops in your solution below is disallowed.


In [3]:
sealed trait Value
case class NumValue(f: Double) extends Value
case object Error extends Value /* -- Do not return Error -- simply throw an new IllegalArgumentException whenever you encounter an erroneous case --*/

type Environment = Map[String, Value]

def evalExpr(e: Expr, env: Environment): Value = {
    
    e match {
        case Const(f) => NumValue(f)
        case Ident(x) => { 
            if (env.contains(x)) { 
                env(x)
            } else {
                throw new IllegalArgumentException("Not found identifier")
            }
        }
        case Plus(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            (v1, v2) match {
                case (NumValue(f1), NumValue(f2)) => NumValue(f1 + f2)
                case _ => throw new IllegalArgumentException("plus failed")
            }
        }
        case Mult(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            (v1, v2) match {
                case (NumValue(f1), NumValue(f2)) => NumValue(f1 * f2)
                case _ => throw new IllegalArgumentException("mult failed")
            }
        }
        /*  Let - Evaluate e1 in current env, bind x to the value, and evaluates e2 in the new env
                Input:
                    - x: String
                    - e1: Expr
                    - e2: Expr
                Algorithm:
                    * Evaluate e1 in the current environment
                    * Bind x to the value in the current environment
                    * Evaluate e2 in the new environment
                Output:
                    Value found by evaluating e2 in the new environment
        */
        case Let(x, e1, e2) => {
            // YOUR CODE HERE
            val v1 = evalExpr(e1, env);
            val newEnv = env + (x -> v1);
            evalExpr(e2, newEnv);
        }
        /*  MultiLet - Evaluate each expression in the list in the current env, bind each x the value, and evaluate e2 in the new env
                Input:
                    - xList: List[String]
                    - eList: List[Expr]
                    - e2: Expr
                Algorithm:
                    * Evaluate each expression in the list in the current environment
                        * This is done by 'zipping' - Zipping is pairs the elements of the two lists
                        * FoldLeft is used to iterate through the zipped list (in this case, the environment is the accumulator)
                    * Bind each x to the corresponding value in the current environment
                        * This is done by adding the pair to the current environment
                        * We then evaluate the next pair in the list (and repeat the process until all pairs are evaluated)
                        * We then update the environment with the new pair
                    * Evaluate e2 in the new environment
                Output:
                    Value found by evaluating e2 in the new environment
        
        */
        case MultiLet(xList, eList, e2) => {
            // YOUR CODE HERE
            val newEnv = (xList zip eList).foldLeft(env) {
                case (currentEnv, (x, e)) => {
                    val v = evalExpr(e, currentEnv);
                    currentEnv + (x -> v);
                }
            }
            evalExpr(e2, newEnv);
        }
    }

}

defined trait Value
defined class NumValue
defined object Error
defined type Environment
defined function evalExpr

In [4]:
//BEGIN TEST
/*
 let (x, y) = (10, 20) in 
    let x = y in 
      x +  x * y
*/
val x = Ident("x")
val y = Ident("y")
val let1 = Let("x", y, Plus(x, Mult(x, y)) )
val mlet1 = MultiLet( List("x", "y"), List(Const(10.0), Const(20.0)), let1)
val v = evalExpr(mlet1, Map.empty)
assert(v == NumValue(420.0), s"Test 1 failed expected: NumValue(420.0), obtained $v")

passed(5)
//END TEST

Tests Passed (5 points)

x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
let1: Let = Let(
  id = "x",
  e1 = Ident(s = "y"),
  e2 = Plus(
    e1 = Ident(s = "x"),
    e2 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "y"))
  )
)
mlet1: MultiLet = MultiLet(
  id = List("x", "y"),
  eList = List(Const(d = 10.0), Const(d = 20.0)),
  e2 = Let(
    id = "x",
    e1 = Ident(s = "y"),
    e2 = Plus(
      e1 = Ident(s = "x"),
      e2 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "y"))
    )
  )
)
v: Value = NumValue(f = 420.0)

In [5]:
//BEGIN TEST
/*
 let (x, y) = (10, x) in 
    let x = y in 
      x +  x * y
*/
val x = Ident("x")
val y = Ident("y")
val let1 = Let("x", y, Plus(x, Mult(x, y)) )
val mlet1 = MultiLet( List("x", "y"), List(Const(10.0), x), let1)
try {
    val v = evalExpr(mlet1, Map.empty)
    assert(false, "Test 2 failed -- your code should detect a usage of x that is out of scope")
} catch {
    case e:IllegalArgumentException => { println("Illegal argument exception caught -- as expected!!") }
    case _ => {println("Wrong type of exception thrown")}
}

passed(5)
//END TEST

cell5.sc:16: This catches all Throwables. If this is really intended, use `case _ : Throwable` to clear this warning.
    case _ => {println("Wrong type of exception thrown")}
         ^


Wrong type of exception thrown
Tests Passed (5 points)

x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
let1: Let = Let(
  id = "x",
  e1 = Ident(s = "y"),
  e2 = Plus(
    e1 = Ident(s = "x"),
    e2 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "y"))
  )
)
mlet1: MultiLet = MultiLet(
  id = List("x", "y"),
  eList = List(Const(d = 10.0), Ident(s = "x")),
  e2 = Let(
    id = "x",
    e1 = Ident(s = "y"),
    e2 = Plus(
      e1 = Ident(s = "x"),
      e2 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "y"))
    )
  )
)

In [6]:
//BEGIN TEST
/*
let (x, y, z, w) = (10, 10, 10, 20 ) in 
  let () = () in 
    let w = w in 
       x *( y + w )
*/

val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val w = Ident("w")
val ten = Const(10.0)
val twenty = Const(20.0)
val innerLet2 = Let("w", w, Mult(x, Plus(y, w)))
val multiLet1 = MultiLet(Nil, Nil, innerLet2)
val e = MultiLet(List("x","y","z","w"), List(ten, ten, ten, twenty), multiLet1)
val v = evalExpr(e, Map.empty)
assert(v == NumValue(300.0), "Test2 Failed -- expected value NumValue(300.0), obtained value $v")

passed(5)
//END TEST

Tests Passed (5 points)

x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
z: Ident = Ident(s = "z")
w: Ident = Ident(s = "w")
ten: Const = Const(d = 10.0)
twenty: Const = Const(d = 20.0)
innerLet2: Let = Let(
  id = "w",
  e1 = Ident(s = "w"),
  e2 = Mult(
    e1 = Ident(s = "x"),
    e2 = Plus(e1 = Ident(s = "y"), e2 = Ident(s = "w"))
  )
)
multiLet1: MultiLet = MultiLet(
  id = List(),
  eList = List(),
  e2 = Let(
    id = "w",
    e1 = Ident(s = "w"),
    e2 = Mult(
      e1 = Ident(s = "x"),
      e2 = Plus(e1 = Ident(s = "y"), e2 = Ident(s = "w"))
    )
  )
)
e: MultiLet = MultiLet(
  id = List("x", "y", "z", "w"),
  eList = List(
    Const(d = 10.0),
    Const(d = 10.0),
    Const(d = 10.0),
    Const(d = 20.0)
  ),
  e2 = MultiLet(
    id = List(),
    eList = List(),
    e2 = Let(
      id = "w",
      e1 = Ident(s = "w"),
      e2 = Mult(
        e1 = Ident(s = "x"),
        e2 = Plus(e1 = Ident(s = "y"), e2 = Ident(s = "w"))
      )
    )
  )
)
v: Value = NumValue(f = 300.0)

## Problem 2: Translating Lettuce Into Scala (25 points)

In this problem, we will translate Lettuce programs into scala. We will consider the fragment of the
language with Let bindings and if-then-else statements.

$$\renewcommand\Expr{\mathbf{Expr}}$$
$$ \begin{array}{rcl}
\Expr &\Rightarrow & \text{Const}(\mathbf{Double}) \\
& | & \text{ConstTrue}\\
&|& \text{ConstFalse}\\
& | & \text{Ident}(\mathbf{String}) \\
& | & \text{Plus}(\Expr, \Expr) \\
&|& \text{Minus}(\Expr, \Expr) \\
& |& \text{Mult}(\Expr, \Expr) \\
& | & \text{Geq}(\Expr, \Expr) \\
& | & \text{And}(\Expr, \Expr) \\
&|& \text{Or}(\Expr, \Expr) \\
& | & \text{IfThenElse}(\Expr, \Expr, \Expr) \\
&|& \text{Let}(\mathbf{String}, \Expr, \Expr) \\
\end{array}$$


$$\newcommand\translate{\textsf{translateIntoScala}}$$ 
The goal is to implement the function  $\translate(e)$ that inputs a Lettuce Expr $e$ and outputs
a __string__ that is supposed to be a scala expression. 

We provide semantics of the translation as below:

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array} (\text{#3})}$$

Note that the output of $\translate$ is a string. 
$$\semRule{}{\translate(\text{Const}(f)) = \textsf{convertToString}(f) }{const}$$

Note that to convert a number to string in scala, use the 'toString' method.

$$\semRule{}{\translate(\text{Ident}(x)) = x }{ident}$$
$$\semRule{}{\translate(\text{ConstTrue}) = "true" }{true}$$
$$\semRule{}{\translate(\text{ConstFalse}) = "false" }{false}$$

Note that when you translate subexpressions, you have to make sure to wrap them in curly braces so that
things defined in the scope of one subexpression do not accidently fall into another. <font color="red"> Please follow the conventions below with curly braces. Otherwise you will not pass the tests </font>

$$\semRule{\translate(e1) = s1,\ \translate(e2) = s2}{\translate(\text{Plus}(e1, e2)) = \{ s1 \} + \{ s2 \} }{plus}$$
$$\semRule{\translate(e1) = s1,\ \translate(e2) = s2}{\translate(\text{Minus}(e1, e2)) = \{ s1  \} - \{ s2 \} }{minus}$$
$$\semRule{\translate(e1) = s1,\ \translate(e2) = s2}{\translate(\text{Geq}(e1, e2)) = \{ s1 \} >= \{ s2 \} }{geq}$$
$$\semRule{\translate(e1) = s1,\ \translate(e2) = s2}{\translate(\text{And}(e1, e2)) = \{ s1  \}\ \&\!\&\ \{ s2 \} }{and}$$
$$\semRule{\translate(e1) = s1,\ \translate(e2) = s2}{\translate(\text{Or}(e1, e2)) = \{ s1 \}\ |\!|\ \{ s2 \} }{or}$$
$$\semRule{\translate(e) = s, \translate(e1) = s1,\ \translate(e2) = s2}{\translate(\text{IfThenEllse}(e,e1, e2)) = \text{if}\ (\{s\})\ \{ s1 \}\ else\ \{ s2 \} }{ite}$$
$$\semRule{\translate(e1) = s1,\ \translate(e2) = s2}{\translate(\text{Let}(x,e1, e2)) =\ \text{val}\ x\ =\ \{ s1 \}\  \{ s2 \}  }{let}$$

Whitespaces (space, tab, returns) are ignored by our test cases. 

In [7]:
sealed trait Expr
case class Const(f: Double) extends Expr
case object ConstTrue extends Expr
case object ConstFalse extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class And(e1: Expr, e2: Expr) extends Expr
case class Or(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x:String, e1: Expr, e2: Expr) extends Expr

defined trait Expr
defined class Const
defined object ConstTrue
defined object ConstFalse
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Geq
defined class And
defined class Or
defined class IfThenElse
defined class Let

In [8]:
/*  translateIntoScala - Translate the given expression into a Scala expression
        Input:
            - e: Expr
        Algorithm:
            * Match the expression with the following cases:
                * Const(f) - Return the double as a string
                * ConstTrue - Return "true"
                * ConstFalse - Return "false"
                * Ident(s) - Return the string
                * Plus(e1, e2) - Return the string "{e1} + {e2}"
                * Minus(e1, e2) - Return the string "{e1} - {e2}"
                * Mult(e1, e2) - Return the string "{e1} * {e2}"
                * Geq(e1, e2) - Return the string "{e1} >= {e2}"
                * And(e1, e2) - Return the string "{e1} && {e2}"
                * Or(e1, e2) - Return the string "{e1} || {e2}"
                * IfThenElse(e, e1, e2) - Return the string "if ({e}) {e1} else {e2}"
                * Let(x, e1, e2) - Return the string "val x = {e1} {e2}"
        Output:
            String representation of the expression in Scala
*/
def translateIntoScala(e: Expr): String = {
    // YOUR CODE HERE
    e match {
        case Const(f) => f.toString
        case ConstTrue => "true"
        case ConstFalse => "false"
        case Ident(x) => x
        case Plus(e1, e2) => s"{${translateIntoScala(e1)}} + {${translateIntoScala(e2)}}"
        case Minus(e1, e2) => s"{${translateIntoScala(e1)}} - {${translateIntoScala(e2)}}"
        case Mult(e1, e2) => s"{${translateIntoScala(e1)}} * {${translateIntoScala(e2)}}"
        case Geq(e1, e2) => s"{${translateIntoScala(e1)}} >= {${translateIntoScala(e2)}}"
        case And(e1, e2) => s"{${translateIntoScala(e1)}} && {${translateIntoScala(e2)}}"
        case Or(e1, e2) => s"{${translateIntoScala(e1)}} || {${translateIntoScala(e2)}}"
        case IfThenElse(e, e1, e2) => s"if ({${translateIntoScala(e)}}) {${translateIntoScala(e1)}} else {${translateIntoScala(e2)}}"
        case Let(x, e1, e2) => s"val $x = {${translateIntoScala(e1)}} {${translateIntoScala(e2)}}"
    }
}

defined function translateIntoScala

In [9]:
// PLEASE RUN THIS CELL BEFORE TESTING FURTHER
def cleanUpWhiteSpacesInString(st: String): String = 
    st.filterNot( _.isWhitespace )

val tst1 = """
val x = { { 1 } + { 3 } }
     {
        val y = { 2 }
        {
            val z = { 10 }
                { x } + { { y } * { z } }
        }
     }

"""
print(cleanUpWhiteSpacesInString(tst1))

def checkWhitespaceMunged(testName: String, s1: String, s2: String) : Boolean = {
    println(s"--- Test $testName ---")
    println("Your code returned:")
    println(s"$s1")
    println("Expected result:")
    println(s"$s2")
    cleanUpWhiteSpacesInString(s1) == cleanUpWhiteSpacesInString(s2)
}

valx={{1}+{3}}{valy={2}{valz={10}{x}+{{y}*{z}}}}

defined function cleanUpWhiteSpacesInString
tst1: String = """
val x = { { 1 } + { 3 } }
     {
        val y = { 2 }
        {
            val z = { 10 }
                { x } + { { y } * { z } }
        }
     }

"""
defined function checkWhitespaceMunged

In [10]:
//BEGIN TEST

val x = Ident("x")
val y = Ident("y")
val one = Const(1.0)
val two = Const(2.0)

val e1 = Plus(x, y)
val e1Expected = "{ x } + { y }"
val e1Translated = translateIntoScala(e1)
assert(checkWhitespaceMunged("e1", e1Translated, e1Expected), "Failed test for e1")
println("Test passed!")

val e2 = Plus(x, one)
val e2Expected = "{ x } + { 1.0 }"
assert(checkWhitespaceMunged("e2", translateIntoScala(e2), e2Expected), "Failed test for e2")
println("Test passed!")

val e3 = Minus(x, two)
val e3Expected = "{ x } - { 2.0 }"
assert(checkWhitespaceMunged("e3", translateIntoScala(e3), e3Expected), "Failed test for e3")
println("Test passed!")

val e4 = Plus(x, Minus(y, one))
val e4Expected = "{ x } + { { y } - {1.0} }"
val s4 = translateIntoScala(e4)
assert(checkWhitespaceMunged("e4", s4, e4Expected), "Failed test for e4")
println("Test passed!")

val e5 = And( Geq( x, y) , Geq(y, one))
val e5Expected = "{{x} >= {y}} && {{y} >= {1.0}}"
val s5 = translateIntoScala(e5)
assert(checkWhitespaceMunged("e5", s5, e5Expected), "Failed test for e5")
println("Test passed!")

val e6 = Or( Geq( Mult(x, y), Ident("z")) , Geq(y, one))
val e6Expected = "{{ {x} * {y}} >= {z}} || {{y} >= {1.0}}"
val s6 = translateIntoScala(e6)
assert(checkWhitespaceMunged("e6", s6, e6Expected), "Failed test for e6")
println("test passed!")

passed(10)

//END TEST

--- Test e1 ---
Your code returned:
{x} + {y}
Expected result:
{ x } + { y }
Test passed!
--- Test e2 ---
Your code returned:
{x} + {1.0}
Expected result:
{ x } + { 1.0 }
Test passed!
--- Test e3 ---
Your code returned:
{x} - {2.0}
Expected result:
{ x } - { 2.0 }
Test passed!
--- Test e4 ---
Your code returned:
{x} + {{y} - {1.0}}
Expected result:
{ x } + { { y } - {1.0} }
Test passed!
--- Test e5 ---
Your code returned:
{{x} >= {y}} && {{y} >= {1.0}}
Expected result:
{{x} >= {y}} && {{y} >= {1.0}}
Test passed!
--- Test e6 ---
Your code returned:
{{{x} * {y}} >= {z}} || {{y} >= {1.0}}
Expected result:
{{ {x} * {y}} >= {z}} || {{y} >= {1.0}}
test passed!
Tests Passed (10 points)

x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
one: Const = Const(f = 1.0)
two: Const = Const(f = 2.0)
e1: Plus = Plus(e1 = Ident(s = "x"), e2 = Ident(s = "y"))
e1Expected: String = "{ x } + { y }"
e1Translated: String = "{x} + {y}"
e2: Plus = Plus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
e2Expected: String = "{ x } + { 1.0 }"
e3: Minus = Minus(e1 = Ident(s = "x"), e2 = Const(f = 2.0))
e3Expected: String = "{ x } - { 2.0 }"
e4: Plus = Plus(
  e1 = Ident(s = "x"),
  e2 = Minus(e1 = Ident(s = "y"), e2 = Const(f = 1.0))
)
e4Expected: String = "{ x } + { { y } - {1.0} }"
s4: String = "{x} + {{y} - {1.0}}"
e5: And = And(
  e1 = Geq(e1 = Ident(s = "x"), e2 = Ident(s = "y")),
  e2 = Geq(e1 = Ident(s = "y"), e2 = Const(f = 1.0))
)
e5Expected: String = "{{x} >= {y}} && {{y} >= {1.0}}"
s5: String = "{{x} >= {y}} && {{y} >= {1.0}}"
e6: Or = Or(
  e1 = Geq(
    e1 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "y")),
    e2 = Ident(s = "z")
  ),
  e2 = Geq(e1 = Ident(s = "y"), e2 = Const(f = 1.0

In [11]:
//BEGIN TEST
val x = Ident("x")
val y = Ident("y")
val one = Const(1.0)
val two = Const(2.0)
val e5 = And( Geq( x, y) , Geq(y, one))
val e5Expected = "{{x} >= {y}} && {{y} >= {1.0}}"
val iteExpr = IfThenElse(e5, one, y)
val iteExpected = s""" if ({$e5Expected}) {
    1.0
} else {
    y
}
"""
val iteTranslated = translateIntoScala(iteExpr)
println("iteTranslated = " + iteTranslated)
assert(checkWhitespaceMunged("itetest", iteTranslated, iteExpected), "Failed test for ite")
passed(5)
//END TEST

iteTranslated = if ({{{x} >= {y}} && {{y} >= {1.0}}}) {1.0} else {y}
--- Test itetest ---
Your code returned:
if ({{{x} >= {y}} && {{y} >= {1.0}}}) {1.0} else {y}
Expected result:
 if ({{{x} >= {y}} && {{y} >= {1.0}}}) {
    1.0
} else {
    y
}

Tests Passed (5 points)

x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
one: Const = Const(f = 1.0)
two: Const = Const(f = 2.0)
e5: And = And(
  e1 = Geq(e1 = Ident(s = "x"), e2 = Ident(s = "y")),
  e2 = Geq(e1 = Ident(s = "y"), e2 = Const(f = 1.0))
)
e5Expected: String = "{{x} >= {y}} && {{y} >= {1.0}}"
iteExpr: IfThenElse = IfThenElse(
  e1 = And(
    e1 = Geq(e1 = Ident(s = "x"), e2 = Ident(s = "y")),
    e2 = Geq(e1 = Ident(s = "y"), e2 = Const(f = 1.0))
  ),
  e2 = Const(f = 1.0),
  e3 = Ident(s = "y")
)
iteExpected: String = """ if ({{{x} >= {y}} && {{y} >= {1.0}}}) {
    1.0
} else {
    y
}
"""
iteTranslated: String = "if ({{{x} >= {y}} && {{y} >= {1.0}}}) {1.0} else {y}"

In [12]:
//BEGIN TEST
val x = Ident("x")
val y = Ident("y")
val one = Const(1.0)
val two = Const(2.0)
val e6 = Or( Geq( Mult(x, y), Ident("z")) , Geq(y, one))
val e6Expected = "{{ {x} * {y}} >= {z}} || {{y} >= {1.0}}"

val letExpr1 = Let("x", Const(10.0) , Plus(x, one))
val letExpr1Expected = """val x = { 10.0 } {
    { x } + { 1.0 } }
"""
val letExprTranslated1 = translateIntoScala(letExpr1)
assert(checkWhitespaceMunged("letExprTranslated1",letExprTranslated1, letExpr1Expected), "Failed test for letExpr1")


val letExpr2 = Let("x", letExpr1 , Let ("y", two, Plus(x, y)))
val letExprTranslated2 = translateIntoScala(letExpr2)
val letExprExpected2 = """ val x = {
                 	   
                val x = {
                 	   10.0
                }
                {
                 	  { x } + {1.0}
                }
            
                }
                {
                 	  
                val y = {
                 	   2.0
                }
                {
                 	  { x } + {y}
                }
            
                }
"""

assert(checkWhitespaceMunged("letExprTranslated2", letExprTranslated2, letExprExpected2), "Failed test for letExpr2")
passed(10)
//END TEST

--- Test letExprTranslated1 ---
Your code returned:
val x = {10.0} {{x} + {1.0}}
Expected result:
val x = { 10.0 } {
    { x } + { 1.0 } }

--- Test letExprTranslated2 ---
Your code returned:
val x = {val x = {10.0} {{x} + {1.0}}} {val y = {2.0} {{x} + {y}}}
Expected result:
 val x = {
                 	   
                val x = {
                 	   10.0
                }
                {
                 	  { x } + {1.0}
                }
            
                }
                {
                 	  
                val y = {
                 	   2.0
                }
                {
                 	  { x } + {y}
                }
            
                }

Tests Passed (10 points)

x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
one: Const = Const(f = 1.0)
two: Const = Const(f = 2.0)
e6: Or = Or(
  e1 = Geq(
    e1 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "y")),
    e2 = Ident(s = "z")
  ),
  e2 = Geq(e1 = Ident(s = "y"), e2 = Const(f = 1.0))
)
e6Expected: String = "{{ {x} * {y}} >= {z}} || {{y} >= {1.0}}"
letExpr1: Let = Let(
  x = "x",
  e1 = Const(f = 10.0),
  e2 = Plus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
)
letExpr1Expected: String = """val x = { 10.0 } {
    { x } + { 1.0 } }
"""
letExprTranslated1: String = "val x = {10.0} {{x} + {1.0}}"
letExpr2: Let = Let(
  x = "x",
  e1 = Let(
    x = "x",
    e1 = Const(f = 10.0),
    e2 = Plus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
  ),
  e2 = Let(
    x = "y",
    e1 = Const(f = 2.0),
    e2 = Plus(e1 = Ident(s = "x"), e2 = Ident(s = "y"))
  )
)
letExprTranslated2: String = "val x = {val x = {10.0} {{x} + {1.0}}} {val y = {2.0} {{x} + {y}}}"
letExprExpected2: String = """ val x = {
                 	   